# Cooperatives Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from dateutil.relativedelta import relativedelta

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
# this will load all the envars from a .env file located in the project root (api)
%load_ext dotenv
load_dotenv(find_dotenv())

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
PG_USER=os.getenv("PG_USER", "")
PG_PASSWORD=os.getenv("PG_PASSWORD", "")
PG_HOST=os.getenv("PG_HOST", "")
PG_PORT=os.getenv("PG_PORT", "5432")
PG_DB_NAME=os.getenv("PG_DB_NAME", "")
PG_SOCKET=os.getenv("PG_SOCKET", "")

if PG_SOCKET != "":
    import pg8000
    connect_to_db = f"postgresql+pg8000://{PG_USER}:{PG_PASSWORD}@/{PG_DB_NAME}?unix_sock={PG_SOCKET}/.s.PGSQL.5432"
else:
    connect_to_db = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB_NAME}"
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql firm_registration_filings  <<
SELECT UPPER(a.city),  REPLACE(a.postal_code,' ',''), count(*)
FROM businesses b, offices o, addresses a, filings f
WHERE b.id=f.business_id
AND b.id=o.business_id
AND o.id=a.office_id
AND a.address_type='mailing'
AND f.filing_type = 'registration'
AND o.office_type = 'businessOffice'
AND to_char(b.founding_date,'yyyymm') = to_char(current_date-27,'yyyymm')
AND b.legal_type in ('SP', 'GP')
GROUP BY upper(a.city),replace(a.postal_code,' ','')
ORDER BY upper(a.city),replace(a.postal_code,' ','');

In [ ]:
last_month = datetime.now() - relativedelta(months=1)
filename = os.path.join(os.getcwd(), r"data/")+"bc_stats_firms_for_" + format(last_month, "%B_%Y") +".csv"

with open(filename, "w") as f:
    f.write("FIRM REGISTRATIONS FOR THE MONTH OF " + format(last_month, "%B_%Y") + " :\n")
    df=firm_registration_filings.DataFrame()
    if not df.empty:
        df.columns = ["City","Postal_Code","Count"]         
    df.to_csv(f, sep=",", encoding="utf-8", index=False)
        
    